# Language Server Protocol (LSP) Integration Example

The [Language Server Protocol](https://microsoft.github.io/language-server-protocol/) is an open standard for communication between development tools and language servers.

Specifically, the [Language Server for Kotlin](https://github.com/Kotlin/kotlin-lsp) provides features that integrate with JetBrains IDEs for Kotlin development.

## Downloading the Language Server for Kotlin

The [Language Server for Kotlin](https://github.com/Kotlin/kotlin-lsp) is available as a standalone distribution.
Follow the instructions on the project page to download the latest version for your operating system.
Make sure the `kotlin-lsp` executable is available on your `PATH`.

## IntelliJ Platform Integration and Bundled Plugin

Declare the IntelliJ Platform integration to configure the necessary libraries.

In [1]:
%use intellij-platform

IntelliJ Platform integration is loaded

The Language Server Protocol support is available in commercial JetBrains IDEs.
To enable it, load the bundled plugin that provides this functionality.
Use the `loadBundledPlugins(pluginIds)` function to load it.

In [2]:
loadBundledPlugins("com.intellij.platform.lsp")

## Integrating the Server with Kotlin Files

To integrate the Kotlin LSP, two components are required:

1. An LSP server descriptor that indicates supported files and provides a command line to start the server in a standalone process.
2. An LSP support provider that ensures the server is available for a given file.

### Server Descriptor

Create a server descriptor that supports files with the `.kt` extension.
To start the LSP server, create a command line that runs the `kotlin-lsp` binary (available on your `PATH`) with the `--stdio` switch, which enables communication over standard input and output streams.

In [3]:
import com.intellij.execution.configurations.GeneralCommandLine
import com.intellij.openapi.project.Project
import com.intellij.openapi.vfs.VirtualFile
import com.intellij.platform.lsp.api.ProjectWideLspServerDescriptor

class KotlinLspServerDescriptor(project: Project) : ProjectWideLspServerDescriptor(project, "KN Kotlin LSP") {
    override fun isSupportedFile(file: VirtualFile) = file.extension == "kt"

    override fun createCommandLine() = GeneralCommandLine("kotlin-lsp", "--stdio")
}

## LSP Support Provider

Create an LSP support provider that starts the LSP server, if necessary, using the corresponding server descriptor.


In [4]:
import com.intellij.platform.lsp.api.LspServerSupportProvider

class KotlinLspServerSupportProvider : LspServerSupportProvider {
    override fun fileOpened(project: Project, file: VirtualFile, serverStarter: LspServerSupportProvider.LspServerStarter) {
        if (file.extension == "kt") {
            serverStarter.ensureServerStarted(KotlinLspServerDescriptor(project))
        }
    }
}

## Registering an Extension Point

`KotlinLspServerSupportProvider` is an implementation that must be registered under the corresponding extension point.
We’ll represent the plugin descriptor extension declaration in Kotlin code.

A full-fledged plugin would declare the following in `plugin.xml`:

```xml
<extensions defaultExtensionNs="com.intellij">
    <platform.lsp.serverSupportProvider implementation="com.example.KotlinLspServerSupportProvider"/>
</extensions>
```

In this Kotlin Notebook, an XML descriptor is not required.
Instead, create an instance of the LSP server support provider and register it under the corresponding extension point.

In [5]:
import com.intellij.platform.lsp.api.LspServerManager
import com.intellij.platform.lsp.api.LspServerSupportProvider

registerExtension(LspServerSupportProvider.EP_NAME, KotlinLspServerSupportProvider())

IntelliJ Platform integration is disposed

## Testing the Integration

To test the integration:
1. Install [Kotlin LSP CLI tool](https://github.com/Kotlin/kotlin-lsp?tab=readme-ov-file#install-kotlin-lsp-cli).
2. Run this Kotlin Notebook.
3. Create a Kotlin file with some unnecessary constructs.

   ```kotlin
   import java.util.List

   class Hello {
       fun sayHello() = "Hello, world!"
   }
   ```
4. Observe two warnings:
   - an unnecessary import,
   - an implicit return type for the `sayHello` function.

Use [Intention Actions](https://www.jetbrains.com/help/idea/intention-actions.html) and note two actions provided by the LSP server:

- *Organize Imports* removes the unnecessary `import java.util.List`.
- *Specify type explicitly* adds an explicit `String` return type to `sayHello`.

These actions may be duplicated by native intention actions provided by JetBrains IDEs such as *Organize imports* or *Specify return type explicitly*.

## Summary

- Load the `com.intellij.modules.ultimate` plugin to enable LSP functionality.
- Register a `LspServerSupportProvider` implementation under the `com.intellij.platform.lsp.serverSupportProvider` extension point.
- Create a `ProjectWideLspServerDescriptor` subclass to launch the LSP server in a standalone process, using standard input and output streams as the communication channel.
- Verify the functionality via the Intention Actions list.